<a href="https://colab.research.google.com/github/VicDc/VIC_/blob/main/shipment_tracking_w_t_CartoPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from geopy.geocoders import Nominatim
import time

# === 1. Load the XML file ===
# Example XML file:
# <shipment>
#    <checkpoint>
#        <location>Wuxi, China</location>
#        <datetime>2025-04-14T16:38</datetime>
#        <status>Collected</status>
#    </checkpoint>

tree = ET.parse('shipment.xml')  # <-- Load shipment.xml
root = tree.getroot()

# === 2. Setup geolocator ===
geolocator = Nominatim(user_agent="shipment_mapper")

# === 3. Extract points using geocoding ===
lons, lats = [], []
labels = []

for checkpoint in root.findall('checkpoint'):
    location = checkpoint.find('location').text
    datetime = checkpoint.find('datetime').text
    status = checkpoint.find('status').text

    # Automatic geocoding
    try:
        geo_location = geolocator.geocode(location)
        if geo_location:
            lon, lat = geo_location.longitude, geo_location.latitude
            lons.append(lon)
            lats.append(lat)
            labels.append(f"{location}\n{datetime}\n{status}")
            print(f"✅ Geocoding '{location}': {lat}, {lon}")
        else:
            print(f"⚠️ Not found: {location}")
    except Exception as e:
        print(f"❌ Geocoding error for {location}: {e}")

    # To avoid overloading the free server
    time.sleep(1)

# === 4. Create the map ===
fig = plt.figure(figsize=(14, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Base cartography
ax.stock_img()
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Draw the route
ax.plot(lons, lats, color='blue', marker='o', markersize=5, linewidth=2, transform=ccrs.Geodetic())

# Add annotations
for lon, lat, label in zip(lons, lats, labels):
    ax.text(lon + 1, lat + 1, label, transform=ccrs.Geodetic(), fontsize=8)

# Dynamic zoom on the route
if lons and lats:
    margin = 10
    ax.set_extent([
        min(lons) - margin, max(lons) + margin,
        min(lats) - margin, max(lats) + margin
    ], crs=ccrs.PlateCarree())

# Title
plt.title('Shipment Route')

# Save the image
plt.savefig('shipment_route_auto_map.png', dpi=300)
plt.close()

print("✅ Automatic geographic map saved as 'shipment_route_auto_map.png'!")


✅ Geocoding 'Wuxi, China': 31.5776626, 120.2952752
✅ Geocoding 'Hong Kong': 22.350627, 114.1849161
✅ Geocoding 'Amsterdam, Netherlands': 52.3730796, 4.8924534
✅ Geocoding 'Rome, Italy': 41.8933203, 12.4829321
✅ Automatic geographic map saved as 'shipment_route_auto_map.png'!
